# Import Costs into S/4HANA
## Background
Costs are displayed in a Global Bike sales order as "Internal Price" with Condition Type VPRS.
We need costs (C08) to be able to compute sensible profit margins later. Since there are no regular cost calculations or goods receipts in Global Bike, we want to manipulate the costs (C08) directly. As a consequence, *this Global Bike client for test data deviates from the standard Global Bike client with respect to costs (and prices)*.

### Source for costs
This costs in a sales order are drawn from table field `MBEW-STPRS` (Standard price), or, `MBEW-VERPR` (Moving average price), resp., depending on the Setting "Price Control" in "Accounting 1" of the material master. Table `MBEWH` contains the historic prices per period.

So for processing, there is just the current price available (although there might be also future prices calculated via CO). Unlike our approach for prices, we cannot define the costs for all past and future years at once. When creating a sales order (for a given year possibly in the past) we have to check/set the *current* costs accordingly first. Note that these current costs are then used for a sales order date in the past.

### Setting the costs
The costs (Standard or Moving price according to the setting in the material master) can be set manually via Transaction `MR21` or `BAPI_MATVAL_PRICE_CHANGE` as long as there are no pending CO-calculations in the way. In the latter case, we get the following error message.

![Error message CKPRCH025](../image/Error%20CKPRCH025.png)

The document for the price change can be viewed using Transaction `CKMPCD`.

If stock exists for the material, a revaluation posting will be created. In regular operation, the costs should be set only once a year.

## Preparations
### Delete CO costs
To avoid the above mentioned error, we have to delete current cost estimates which exist in the Global Bike client. This is done via Transaction `CKR1` using the following settings.

![Transaction CKR1.png](../image/Transaction%20CKR1.png)

### Allow cost changes
Wenn trying to change the cost via Transaction `MR21`, we get the error
![](../image/CKMLSENDPRICE701.png)

To allow cost changes, we have to add a configuration for each of our five plants via View `VCKML_PRICE_SEND` like so:
![](../image/VCKML_PRICE_SEND.png)

## Implementation
### TSV file for importing costs to S4
We first have to prepare the costs from `generator/masterdata.xls` and import them into a DB table in S4.

In [1]:
import pandas as pd
import re as re

In [2]:
masterdata = '../generator/masterdata.xlsx'
costs_eur = pd.read_excel(masterdata, sheet_name="costs EUR")
costs_usd = pd.read_excel(masterdata, sheet_name="costs USD")
costs_eur.sample()

,MATNR,MATKL,Unnamed: 2,Marge,MAKTX,2020,2021,2022,2023,2024,...,2026,2027,2028,2029,2030,2031,2032,2033,2034,2035
4,DGRW2000,UTIL,ACC,0.5,Deluxe GPS-Bike Computer Silver White,195.0,197.0,206.6,225.3,234.3,...,246.2,258.2,281.6,292.9,304.6,307.8,322.9,352.1,366.2,380.8


We need only the material and year-columns

In [3]:
columns = [
    'MATNR', 
    *costs_eur.columns.astype(str).str.extract(r'(20\d\d)')[0].dropna().astype(int).to_list()
]

Convert costs to tidy format and add sales organizations

In [4]:
costs = (
    # combine both tables
    pd.concat([costs_eur[columns], costs_usd[columns]], keys=["EUR", "USD"], names=["Currency"])
    .reset_index("Currency")
    # tidy
    .melt(id_vars=["MATNR", "Currency"], var_name="Year", value_name="Cost")
    # for each currency we have two sales organizations
    .assign(plants=lambda df: df["Currency"].map({'EUR':'HD00,HH00', 'USD':'DL00,MI00,SD00'}))
    .assign(Plant=lambda df: df["plants"].str.split(','))
)
costs2 = (
  costs.explode("Plant")
  .reindex(columns=["Plant", "MATNR", "Year", "Cost", "Currency"])
)
costs2.sample()

,Plant,MATNR,Year,Cost,Currency
0,HH00,BOTL1000,2020,10.0,EUR


In [5]:
file = '../data/costs.tsv'
costs2.to_csv(file, index=False, header=False, sep='\t')

### Importing TSV to DB table in S4

We are using ABAP artefacts as described in `Import2ERP.md` as well as these additionally created ones:
- DB table `ZUCC_ANALY_COSTS` "Imported Costs for setting the material internal price"

| Field | Data element |   |
|-------|--------------|---|
| MANDT	| MANDT | key
| PLANT | WERKS_D | key
| MATNR | MATNR | key
| SO_YEAR | NUMC(4) | year of the sales order
| COST | STPRS | Reference field: WAERS
| WAERS | WAERS |

- Structure `ZUCC_ANALYTICS_COSTS_READ` with the same fields as before, but without `MANDT`


Program `ZUCC_ANALYTICS_SDGEN_IMPCOSTS` then does the import.

### Setting the actual costs in S4

Function `ZUCC_ANALYTICS_SDGEN_COST` in function group `ZUCC_ANALYTICS_SDGEN`
sets the actual cost according to a given year for each known plant and material.

Sine we expect the costs to stay constant during a year, the function should be silent and efficient in most cases.
Although the currency is derived from the plant, we have to provide a currency.